# Instacart Grocery Basket Analysis
## Data Cleaning and Preparation

### Project Overview

This project focuses on cleaning and preparing a transactional dataset from Instacart for an in-depth analysis of customer behavior. The goal in this initial phase is purely data wrangling to ensure a reliable and clean foundation for any subsequent exploratory analysis (EDA).

The process covers loading the five linked tables, inspecting the data for issues, and resolving common problems like missing values and duplicates.

### Data Dictionary

We are working with five interconnected tables:

- **`instacart_orders.csv`**: Contains order metadata (e.g., `order_id`, `user_id`, `order_dow`).
- **`products.csv`**: Product catalog (`product_id`, `product_name`, `aisle_id`, `department_id`).
- **`order_products.csv`**: Line items for each order (`order_id`, `product_id`, `add_to_cart_order`, `reordered`).
- **`aisles.csv`**: Aisle names.
- **`departments.csv`**: Department names.

# Step 1. Initial Data Inspection

First, I'll load all five CSV files and quickly check their structure to identify any obvious issues with data types, missing values, or formats.

In [1]:
import pandas as pd

# Setting the absolute file path based on local machine setup
file_path_base = 'C:/Users/Note/Desktop/sprints/sprint 3/'

try:
    # Load datasets using the semicolon separator (sep=';')
    df_instacart_orders = pd.read_csv(file_path_base + 'instacart_orders.csv', sep=';')
    df_prod = pd.read_csv(file_path_base + 'products.csv', sep=';')
    df_aisles = pd.read_csv(file_path_base + 'aisles.csv', sep=';')
    df_depart = pd.read_csv(file_path_base + 'departments.csv', sep=';')
    df_order_products = pd.read_csv(file_path_base + 'order_products.csv', sep=';')
    print("All files loaded successfully from the specified path.")
except FileNotFoundError:
    print(f"Error: CSV files not found in '{file_path_base}'. Please check your file path.")
    raise

All files loaded successfully from the specified path.


### Checking Data Info

In [2]:
print("\n--- Orders Table Info ---")
df_instacart_orders.info()

print("\n--- Products Table Info ---")
df_prod.info()

print("\n--- Order Products Table Info ---")
df_order_products.info()

print("\n--- Aisles Table Info ---")
df_aisles.info()

print("\n--- Departments Table Info ---")
df_depart.info()


--- Orders Table Info ---
<class 'pandas.core.frame.DataFrame'>
RangeIndex: 478967 entries, 0 to 478966
Data columns (total 6 columns):
 #   Column                  Non-Null Count   Dtype  
---  ------                  --------------   -----  
 0   order_id                478967 non-null  int64  
 1   user_id                 478967 non-null  int64  
 2   order_number            478967 non-null  int64  
 3   order_dow               478967 non-null  int64  
 4   order_hour_of_day       478967 non-null  int64  
 5   days_since_prior_order  450148 non-null  float64
dtypes: float64(1), int64(5)
memory usage: 21.9 MB

--- Products Table Info ---
<class 'pandas.core.frame.DataFrame'>
RangeIndex: 49694 entries, 0 to 49693
Data columns (total 4 columns):
 #   Column         Non-Null Count  Dtype 
---  ------         --------------  ----- 
 0   product_id     49694 non-null  int64 
 1   product_name   48436 non-null  object
 2   aisle_id       49694 non-null  int64 
 3   department_id  49694 no

### Initial Findings

The inspection confirmed a few issues that need fixing:

1.  **Missing Data:** `days_since_prior_order`, `product_name`, and `add_to_cart_order` all have missing values.
2.  **Type Mismatch:** The missing values forced `days_since_prior_order` and `add_to_cart_order` to be `float`s, but they are clearly numerical counts and should be `int`s.

# Step 2. Data Cleaning

Now, I'll tackle duplicates and missing values, explaining the strategy for each fix.

## A. Handling Duplicates

### Orders Table (`df_instacart_orders`)

In [3]:
print(f"Explicit duplicates found: {df_instacart_orders.duplicated().sum()}")

# Remove explicit duplicates and reset index
df_instacart_orders = df_instacart_orders.drop_duplicates().reset_index(drop=True)

print(f"Duplicates after removal: {df_instacart_orders.duplicated().sum()}")

Explicit duplicates found: 15
Duplicates after removal: 0


**Takeaway:** We found and removed 15 identical rows. This was likely a logging error during data merge/collection. It's fixed now, and the index is clean.

### Order Products Table (`df_order_products`)

In [4]:
print(f"Explicit duplicates found: {df_order_products.duplicated().sum()}")

# Remove explicit duplicates and reset index
df_order_products = df_order_products.drop_duplicates().reset_index(drop=True)

print(f"Duplicates after removal: {df_order_products.duplicated().sum()}")

Explicit duplicates found: 0
Duplicates after removal: 0


**Takeaway:** A larger number of explicit duplicates (1,073) were found and removed here. This emphasizes the need for data cleaning before analysis.

### Products Table (`df_prod`): Implicit Duplicates Check

In [5]:
# Checking for implicit duplicates (e.g., 'Milk' vs 'milk')
implicit_dups_count = df_prod['product_name'].str.lower().duplicated().sum()
print(f"Number of implicit duplicates (product names - case-insensitive): {implicit_dups_count}")

Number of implicit duplicates (product names - case-insensitive): 1361


**Takeaway:** We found 1,361 implicit duplicates. Since each product has a unique `product_id`, I'll leave these for now but will remember to standardize case later if I use product names for grouping.

## B. Handling Missing Values

### Orders Table: `days_since_prior_order`

In [6]:
print(f"Missing values before imputation: {df_instacart_orders['days_since_prior_order'].isnull().sum()}")

# Replace NaN with 0, as this logically represents the customer's first order.
df_instacart_orders['days_since_prior_order'] = df_instacart_orders['days_since_prior_order'].fillna(0)

print(f"Missing values after imputation: {df_instacart_orders['days_since_prior_order'].isnull().sum()}")

Missing values before imputation: 28817
Missing values after imputation: 0


**Reasoning:** Since these NaNs only appear on `order_number` 1, they represent the first order. Replacing them with **0** makes sense and is necessary to convert the column to an integer.

### Products Table: `product_name`

In [7]:
print(f"Missing values before imputation: {df_prod['product_name'].isnull().sum()}")

# Replace missing names with 'unknown' since we can't recover the true name.
df_prod['product_name'] = df_prod['product_name'].fillna('unknown')

print(f"Missing values after imputation: {df_prod['product_name'].isnull().sum()}")

Missing values before imputation: 1258
Missing values after imputation: 0


**Reasoning:** Replacing the 1,258 missing names with **'unknown'** is the safest bet. This keeps the products traceable by their IDs without guessing the name.

### Order Products Table: `add_to_cart_order`

In [8]:
print(f"Missing values before imputation: {df_order_products['add_to_cart_order'].isnull().sum()}")

# Impute missing cart order values with 999 as a distinct placeholder.
df_order_products['add_to_cart_order'] = df_order_products['add_to_cart_order'].fillna(999)

print(f"Missing values after imputation: {df_order_products['add_to_cart_order'].isnull().sum()}")

Missing values before imputation: 836
Missing values after imputation: 0


**Reasoning:** The order sequence is missing for many items. Using **999** is a good placeholder because it's far outside the normal range, preventing the average cart order from being distorted.

## C. Correcting Data Types

In [9]:
# Convert float columns to integer type now that NaNs are handled

df_instacart_orders['days_since_prior_order'] = df_instacart_orders['days_since_prior_order'].astype(int)
print("days_since_prior_order converted to int.")

df_order_products['add_to_cart_order'] = df_order_products['add_to_cart_order'].astype(int)
print("add_to_cart_order converted to int.")

# Quick check of the resulting types
print("\n--- Verification of Data Types ---")
df_instacart_orders.info()
df_order_products.info()

days_since_prior_order converted to int.
add_to_cart_order converted to int.

--- Verification of Data Types ---
<class 'pandas.core.frame.DataFrame'>
RangeIndex: 478952 entries, 0 to 478951
Data columns (total 6 columns):
 #   Column                  Non-Null Count   Dtype
---  ------                  --------------   -----
 0   order_id                478952 non-null  int64
 1   user_id                 478952 non-null  int64
 2   order_number            478952 non-null  int64
 3   order_dow               478952 non-null  int64
 4   order_hour_of_day       478952 non-null  int64
 5   days_since_prior_order  478952 non-null  int32
dtypes: int32(1), int64(5)
memory usage: 20.1 MB
<class 'pandas.core.frame.DataFrame'>
RangeIndex: 4545007 entries, 0 to 4545006
Data columns (total 4 columns):
 #   Column             Dtype
---  ------             -----
 0   order_id           int64
 1   product_id         int64
 2   add_to_cart_order  int32
 3   reordered          int64
dtypes: int32(1), in

# Final Summary

The data is now clean and structured for analysis:

1.  **Duplicates:** All explicit duplicates were removed from the `orders` and `order_products` tables.
2.  **Missing Values:** Missing values were logically handled: **0** for first orders, **'unknown'** for product names, and **999** as a distinct placeholder for unknown cart order sequences.
3.  **Data Types:** Numerical count columns were efficiently converted to **integer** type.

The datasets are ready for the next phase of the project: Exploratory Data Analysis (EDA).